In [1]:
# !pip install bs4

# Imports
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import re

%matplotlib inline

This is the result of "copy selector" for 
`#main > div:nth-child(2) > p:nth-child(2) > a:nth-child(1)`

```
<a href="/Content/Uploads/Bolivar Medical/files/640923090_Bolivar_Medical_Center_Standard_Charges_12192020.zip">here</a>
```
This the result of "copy selector" for
`#Content_C002_Col00 > article > aside > div.col-xs-12.fifth-child.additional-info > p:nth-child(2) > a`

```
<a href="http://www.mmclc.org/">Visit Hospital Website</a>
```

Inline-style: 
![alt text](https://github.com/nestauk/im-tutorials/blob/3-ysi-tutorial/figures/Web-Scraping/scrape-pipeline.png?raw=1 "workflow")

In [25]:
hosp_url=visit_hospital[0]+"understanding-billing-and-charges"

# Use requests to retrieve data from a given URL
hosp_res=requests.get(hosp_url)

# Parse the whole HTML page using BeautifulSoup
hosp_soup = BeautifulSoup(hosp_res.text, 'html.parser')

# Title of the parsed page
hosp_soup.title

BeautifulSoup(hosp_res.text, 'html.parser').find

# Find all links
# links = [link.get('href') for link in hosp_soup.find_all('a')]
# links_billing = [a['href'] for a in hosp_soup.find_all('a', href=True) if a.text=='here']
links_billing = [a['href'] for a in hosp_soup.find_all('a', href=True) if a['href'].endswith('.zip')]
(links_billing)


list

In [38]:
lpurl=r"http://www.lifepointhealth.net/making-communities-healthier/communities/bolivar-medical-center?loc=c1c283fd-ba54-4ef9-b71e-ebabeda3bea7"

def get_hosp_url(url):
    # Use requests to retrieve data from a given URL
    lp_res=requests.get(url)

    # Parse the whole HTML page using BeautifulSoup
    lp_soup = BeautifulSoup(lp_res.text, 'html.parser')

    # Find all links
    visit_hospital = [a['href'] for a in lp_soup.find_all('a', href=True) if a.text=='Visit Hospital Website']
    
    # check if there is stuff in there
    if len(visit_hospital)>0:
        new_url = visit_hospital[0]
    else:
        new_url=np.nan
    
    return new_url
    
# Add homepage and keep the unique links
# fixed_links = set([''.join([lpurl, link]) for link in links if link])
# fixed_links

In [50]:
def get_zip_url(url):
    """gets link for zip file

    Args:
        url (string): link to webpage containing standard charges and billing information

    Returns:
        [type]: zip file containing xml of standard charges
    """
    
    try:
        # create link based on pattern among lifepoint stuff
        # hosp_url=url+"understanding-billing-and-charges"
        hosp_url=url
        # Use requests to retrieve data from a given URL
        hosp_res=requests.get(hosp_url)

        # Parse the whole HTML page using BeautifulSoup
        hosp_soup = BeautifulSoup(hosp_res.text, 'html.parser')

        # Title of the parsed page
        hosp_soup.title

        BeautifulSoup(hosp_res.text, 'html.parser').find

        # Find all links
        # links = [link.get('href') for link in hosp_soup.find_all('a')]
        # links_billing = [a['href'] for a in hosp_soup.find_all('a', href=True) if a.text=='here']
        links_billing = [a['href'] for a in hosp_soup.find_all('a', href=True) if (a['href'].endswith('.zip') | a['href'].endswith('.xml'))]
        
        # check if there is stuff in there
        if len(links_billing)>0:
            new_url = links_billing[0]
        # elif:
        #     new_url = [a['href'] for a in hosp_soup.find_all('a', href=True) if a['href'].endswith('.xlsx')][0]
        else:
            new_url=np.nan
    except:
        new_url=np.nan
    
    return new_url


In [22]:
# just fetch the zip files
# filtered_list= list(filter(lambda x: x.endswith('.zip'), links_billing))


['/Content/Uploads/Bolivar Medical/files/640923090_Bolivar_Medical_Center_Standard_Charges_12192020.zip']


In [51]:
# !pip install openpyxl
# import website links
lp = pd.read_excel(r"C:\Users\fedde\hospital-price-transparency-v2\lifepoint_free_hosp.xlsx",sheet_name='links')
lp.columns=lp.columns.str.replace(" ","_",regex=False).str.replace("?","",regex=False)

print(lp.info())
lp.head()
lp.columns=lp.columns.str.replace(" ","_",regex=False).str.replace("?","",regex=False)
lp['website2']=lp['full_redirect_link'].apply(get_hosp_url)
lp['zip_url']=lp['charge_page'].apply(get_zip_url)
lp.zip_url.notna().sum()
lp.to_csv("lp_links_webscrape.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   full redirect link  86 non-null     object 
 1   hospital?           86 non-null     bool   
 2   npi                 35 non-null     float64
 3   website             86 non-null     object 
 4   charge page         86 non-null     object 
 5   name                86 non-null     object 
 6   notes               3 non-null      object 
 7   onetab              34 non-null     object 
 8   Unnamed:_9          22 non-null     object 
 9   Unnamed:_10         13 non-null     object 
 10  website2            76 non-null     object 
 11  zip_url             55 non-null     object 
dtypes: bool(1), float64(1), object(10)
memory usage: 7.6+ KB
None


,full redirect link,hospital?,npi,website,charge page,name,notes,onetab,Unnamed:_9,Unnamed:_10,website2,zip_url
0,http://www.lifepointhealth.net/making-communit...,True,1.821078e+09,https://www.andalusiahealth.com/,https://www.andalusiahealth.com/understanding-...,andalusia-regional-hospital,NaN,NaN,NaN,NaN,http://www.andalusiahealth.com/,/Content/Uploads/Andalusia Health/images/63079...
1,http://www.lifepointhealth.net/making-communit...,True,1.457321e+09,https://www.ashleyregional.com/,https://www.ashleyregional.com/understanding-b...,ashley-regional-medical-center,NaN,NaN,NaN,NaN,http://www.avmc-hospital.org/,NaN
2,http://www.lifepointhealth.net/making-communit...,True,1.588745e+09,https://www.bluegrasscommunityhospital.com/,https://www.bluegrasscommunityhospital.com/und...,bluegrass-community-hospital,NaN,NaN,NaN,NaN,http://www.bluegrasscommunityhospital.com/,/sites/bluegrass/assets/uploads/522260534_Blue...
3,http://www.lifepointhealth.net/making-communit...,True,1.457321e+09,https://www.bolivarmedical.com/,https://www.bolivarmedical.com/understanding-b...,bolivar-medical-center,NaN,https://www.bolivarmedical.com/,Bolivar Medical Center,Cleveland MS Hospital,http://www.bolivarmedical.com/,/Content/Uploads/Bolivar Medical/files/6409230...
4,http://www.lifepointhealth.net/making-communit...,True,1.427132e+09,https://www.bourbonhospital.com/,https://www.bourbonhospital.com/understanding-...,bourbon-community-hospital,NaN,https://www.bourbonhospital.com/,Bourbon Community Hospital,NaN,http://www.bourbonhospital.com/,/sites/bourbon/assets/uploads/621757924_Bourbo...


# prime health services

In [3]:
def get_url_by_filetype(url,filetype, backup='.zip'):
    """gets link for  file

    Args:
        url (string): link to webpage containing standard charges and billing information

    Returns:
        [type]:  url of file containing standard charges
    """
    try:
        # create link based on pattern among lifepoint stuff
        # hosp_url=url+"understanding-billing-and-charges"
        hosp_url=url
        # Use requests to retrieve data from a given URL
        hosp_res=requests.get(hosp_url)

        # Parse the whole HTML page using BeautifulSoup
        hosp_soup = BeautifulSoup(hosp_res.text, 'html.parser')

        # Title of the parsed page
        hosp_soup.title

        BeautifulSoup(hosp_res.text, 'html.parser').find

        # Find all links
        # links = [link.get('href') for link in hosp_soup.find_all('a')]
        # links_billing = [a['href'] for a in hosp_soup.find_all('a', href=True) if a.text=='here']
        links_billing = [a['href'] for a in hosp_soup.find_all('a', href=True) if (a['href'].endswith(filetype) | a['href'].endswith(backup))]
        
        # check if there is stuff in there
        if len(links_billing)>0:
            new_url = links_billing[0]
        # elif:
            # new_url = [a['href'] for a in hosp_soup.find_all('a', href=True) if a['href'].endswith(backup)][0]
        else:
            new_url=np.nan
    except:
        new_url=np.nan
    
    return new_url


In [4]:
# !pip install openpyxl
# import website links
file_name = r"prime.xlsx"
sheet = 'links'
prime = pd.read_excel(file_name,sheet_name=sheet)
prime.columns=prime.columns.str.replace(" ","_",regex=False).str.replace("?","",regex=False)


# prime['website2']=prime['full_redirect_link'].apply(get_hosp_url)
prime['file_url']=prime['charge_page'].apply(lambda x: get_url_by_filetype(x, '.json','.zip'))
print("not nas: "+str(prime.file_url.notna().sum()))
prime.to_csv("prime_links_webscrape.csv")

not nas: 27


In [ ]:
def get_url_by_filetype(url,suffixes,filetype, backup='.zip'):
    """gets link for  file

    Args:
        url (string): link to webpage containing standard charges and billing information

    Returns:
        [type]:  url of file containing standard charges
    """
    for suffix in suffixes:
        try:
            # create link based on pattern among lifepoint stuff
            # hosp_url=url+"understanding-billing-and-charges"
            hosp_url=url
            # Use requests to retrieve data from a given URL
            hosp_res=requests.get(hosp_url)

            # Parse the whole HTML page using BeautifulSoup
            hosp_soup = BeautifulSoup(hosp_res.text, 'html.parser')

            # Title of the parsed page
            hosp_soup.title

            BeautifulSoup(hosp_res.text, 'html.parser').find

            # Find all links
            # links = [link.get('href') for link in hosp_soup.find_all('a')]
            # links_billing = [a['href'] for a in hosp_soup.find_all('a', href=True) if a.text=='here']
            links_billing = [a['href'] for a in hosp_soup.find_all('a', href=True) if (a['href'].endswith(filetype) | a['href'].endswith(backup))]
            
            # check if there is stuff in there
            if len(links_billing)>0:
                new_url = links_billing[0]
            # elif:
                # new_url = [a['href'] for a in hosp_soup.find_all('a', href=True) if a['href'].endswith(backup)][0]
            else:
                new_url=np.nan
        except:
            new_url=np.nan
        
    return new_url
